# Basic Server Code

In [1]:
from flask import Flask
from flask import request
import json
import requests
import hashlib as hasher
import datetime as date

In [2]:
node = Flask(__name__)

In [3]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hash_block() 
    def hash_block(self):
        sha = hasher.sha256()
        sha.update(str(self.index) + str(self.timestamp) + str(self.data) + str(self.previous_hash))
        return sha.hexdigest()

In [4]:
def create_genesis_block():
    return Block(0, date.datetime.now(), {"proof-of-work": 1,"transactions": None}, "0")

In [5]:
miner_address = "miner"
# This node's blockchain copy
blockchain = []
blockchain.append(create_genesis_block())

In [6]:
this_nodes_transactions = []
peer_nodes = [] #to be expanded over many nodes in a p2p network
mining = True

In [7]:
@node.route('/txion', methods=['POST'])
def transaction():
    # On each new POST request, extract the transaction data
    new_txion = request.get_json()
    this_nodes_transactions.append(new_txion)
    # Log it to console
    print "New transaction"
    #print "FROM: {}".format(new_txion['from'].encode('ascii','replace'))
    #print "TO: {}".format(new_txion['to'].encode('ascii','replace'))
    print "DATA: {}\n".format(new_txion['data'])
    
    return "Transaction successful\n"

In [8]:
@node.route('/blocks', methods=['GET'])
def get_blocks():
    chain_to_send = blockchain
    # Convert our blocks into dictionaries so we can send them as json objects later
    for i in range(len(chain_to_send)):
        block = chain_to_send[i]
        block_index = str(block.index)
        block_timestamp = str(block.timestamp)
        block_data = str(block.data)
        block_hash = block.hash
        chain_to_send[i] = {
          "index": block_index,
          "timestamp": block_timestamp,
          "data": block_data,
          "hash": block_hash
        }
    chain_to_send = json.dumps(chain_to_send)
    return chain_to_send

def find_new_chains():
    # Get the blockchains of every other node
    other_chains = []
    for node_url in peer_nodes:
        block = requests.get(node_url + "/blocks").content
        block = json.loads(block)
        other_chains.append(block)
    return other_chains

def consensus():
    other_chains = find_new_chains()
    longest_chain = blockchain
    for chain in other_chains:
        if len(longest_chain) < len(chain):
            longest_chain = chain
    blockchain = longest_chain

def proof_of_work(last_proof):
    incrementor = last_proof + 1
    # Keep incrementing the incrementor until
    # it's equal to a number divisible by 9
    # and the proof of work of the previous
    # block in the chain
    while not (incrementor % 999999 == 0 and incrementor % last_proof == 0):
        incrementor += 1
        # Once that number is found,
        # we can return it as a proof
        # of our work
    return incrementor/999999 + last_proof

In [9]:
@node.route('/mine', methods = ['GET'])
def mine():
    last_block = blockchain[len(blockchain) - 1]
    last_proof = last_block.data['proof-of-work']
    proof = proof_of_work(last_proof)
      # we reward the miner by adding a transaction "from": "network", "to": miner_address, 
    this_nodes_transactions.append({ "data": "Transaction mined by miner" })
      # Now we can gather the data needed
      # to create the new block
    new_block_data = {
        "proof-of-work": proof,
        "transactions": list(this_nodes_transactions)
      }
    new_block_index = last_block.index + 1
    new_block_timestamp = this_timestamp = date.datetime.now()
    last_block_hash = last_block.hash
    # Empty transaction list
    this_nodes_transactions[:] = []
    mined_block = Block(new_block_index,new_block_timestamp,new_block_data,last_block_hash)
    blockchain.append(mined_block)
    # Let the client know we mined a block
    return json.dumps({
          "index": new_block_index,
          "timestamp": str(new_block_timestamp),
          "data": new_block_data,
          "hash": last_block_hash
      }) + "\n"



In [10]:
node.run()
#node.run(host='0.0.0.0') #mention IP address in a p2p network

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Feb/2018 14:34:31] "POST /txion HTTP/1.1" 200 -


New transaction
DATA: Hello, this is message 1



127.0.0.1 - - [08/Feb/2018 14:34:33] "POST /txion HTTP/1.1" 200 -


New transaction
DATA: Hello, this is message 2



127.0.0.1 - - [08/Feb/2018 14:34:34] "POST /txion HTTP/1.1" 200 -


...

127.0.0.1 - - [08/Feb/2018 14:34:45] "POST /txion HTTP/1.1" 200 -


New transaction
DATA: Hello, this is message 6



127.0.0.1 - - [08/Feb/2018 14:34:48] "POST /txion HTTP/1.1" 200 -


New transaction
DATA: Hello, this is message 7



127.0.0.1 - - [08/Feb/2018 14:34:50] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2018 14:34:57] "GET /blocks HTTP/1.1" 200 -


# Add transaction command:
# curl "localhost:5000/txion"      -H "Content-Type: application/json"      -d '{"data": "Hello, this is message 1"}'


# mining 
# curl "localhost:5000/mine"

# blocks to be viewed
# curl "localhost:5000/blocks"